In [ ]:
from os import listdir
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from PIL import Image
from kagglehub import dataset_download
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img

In [ ]:
dataset_path = dataset_download("abhinavnayak/catsvdogs-transformed")
print("Path to dataset files:", dataset_path)

In [ ]:
data_path: str = f"{dataset_path}/train_transformed/"
print(data_path)

In [ ]:
list_of_images = os.listdir(data_path)

In [ ]:
number_of_images: int = len(list_of_images)
print(number_of_images)

In [ ]:
i = 1
plt.figure(figsize=(10, 10))
for image in list_of_images:
    if i > 5:
        break
    img = load_img(data_path + image)
    plt.subplot(1, 5, i)
    plt.imshow(img)
    plt.title(str(img.size))
    plt.axis('off')
    i += 1

plt.show()

In [ ]:
images = np.zeros((number_of_images, 224, 224, 3))
labels = np.zeros((number_of_images, 1))

In [ ]:
for image_number, image_path in enumerate(list_of_images):
    img = Image.open(data_path + image_path)
    img = np.asarray(img)
    img = img / img.max()
    images[image_number] = img
    labels[image_number] = 0 if "cat" in image_path else 1

In [ ]:
print(labels)

In [ ]:
labels = keras.utils.to_categorical(labels)

In [ ]:
print(labels)

In [ ]:
images.shape

In [ ]:
plt.imshow(images[0])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model = keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
)

# Freeze the pre-trained layers
for layer in model.layers:
    layer.trainable = False

# Add layers using calls:
x = model.output  # Get the output of the InceptionV3 base
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(512, activation="relu")(x)
predictions = keras.layers.Dense(2, activation="softmax")(x)

# Create a new Functional model with these layers:
model = keras.Model(inputs=model.input, outputs=predictions)

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(x_train, y_train, epochs=50)

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["loss"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_predict = model.predict(x_test)

In [ ]:
y_predict = np.argmax(y_predict, axis=1)
y_test = np.argmax(y_test, axis=1)

In [ ]:
cm = confusion_matrix(y_test, y_predict)
sns.heatmap(cm, annot=True, fmt="d")

In [ ]:
cr = classification_report(y_test, y_predict)
print(cr)

# Model From Scratch

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,  # Degree range for random rotations
    width_shift_range=0.2,  # Fraction of total width for horizontal shifts
    height_shift_range=0.2,  # Fraction of total height for vertical shifts
    shear_range=0.2,  # Shear intensity (shear angle in counter-clockwise direction in degrees)
    zoom_range=0.2,  # Range for random zoom
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Strategy for filling newly created pixels
)

train_generator = datagen.flow(
    images, labels, batch_size=32
)

In [ ]:
train_generator

In [ ]:
images = np.array(images)

In [ ]:
images.shape

In [ ]:
def inception_module(X, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5,
                     filters_pool_proj, name=None):
    conv_1x1 = keras.layers.Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(X)

    conv_3x3 = keras.layers.Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(X)
    conv_3x3 = keras.layers.Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)

    conv_5x5 = keras.layers.Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(X)
    conv_5x5 = keras.layers.Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)

    pool_proj = keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='same')(X)
    pool_proj = keras.layers.Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)

    output = keras.layers.concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)

    return output

In [ ]:
input_layer = tf.keras.layers.Input(shape=(224, 224, 3))

x = tf.keras.layers.Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(
    input_layer)
x = tf.keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = tf.keras.layers.Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = tf.keras.layers.Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = tf.keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = tf.keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')

x1 = tf.keras.layers.AveragePooling2D((5, 5), strides=3)(x)
x1 = tf.keras.layers.Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = tf.keras.layers.Flatten()(x1)
x1 = tf.keras.layers.Dense(1024, activation='relu')(x1)
x1 = tf.keras.layers.Dropout(0.7)(x1)
x1 = tf.keras.layers.Dense(2, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')

x2 = tf.keras.layers.AveragePooling2D((5, 5), strides=3)(x)
x2 = tf.keras.layers.Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = tf.keras.layers.Flatten()(x2)
x2 = tf.keras.layers.Dense(1024, activation='relu')(x2)
x2 = tf.keras.layers.Dropout(0.7)(x2)
x2 = tf.keras.layers.Dense(2, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = tf.keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(2, activation='softmax', name='output')(x)

In [ ]:
model = keras.Model(input_layer, [x, x1, x2], name='inception_v1')

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
y_train.shape

In [ ]:
history = model.fit(train_generator, epochs=50)

In [ ]:
plt.plot(history.history["loss"])
plt.title("Model Loss/Accuracy")
plt.ylabel("Loss/Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
y_test = keras.utils.to_categorical(y_test)

In [ ]:
y_test.shape

In [ ]:
y_test = np.argmax(y_test, axis=1)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
x_test.shape, y_test.shape

In [ ]:
y_predict = model.predict(x_test)

In [ ]:
y_predict = np.array(y_predict)

In [ ]:
y_predict[0].shape

In [ ]:
y_predict = np.argmax(y_predict[0], axis=1)

In [ ]:
y_test.shape, y_predict.shape

In [ ]:
cm = confusion_matrix(y_test, y_predict)
sns.heatmap(cm, annot=True, fmt="d")

In [ ]:
cr = classification_report(y_test, y_predict)
print(cr)